<a href="https://colab.research.google.com/github/limontec/Streamlink-Recorder/blob/master/Streamlink_Recorder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##GDrive access your drive content

In [ ]:
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Mount Gdrive to /content/drive</h3></center><br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive! 
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

##Streamlink

In [ ]:
#@markdown <center><h2>Streamlink Installer</h2></center><br>
#@markdown <center>Install StreamLink</center>
!pip install streamlink

In [ ]:
#@markdown <center><h3><h2>Stream Recorder</h2>Record any stream</h3></center><br>
#@markdown 1- Go to your Google Drive and create a directory called "stream" (whitout quotes).
#@markdown <br />2- Mout Gdrive above.
#@markdown <br />3- Install Streamlink above.
#@markdown <br />4- Start recording bellow.
import os
import time
import subprocess
from IPython.display import clear_output
RUN_Dir = "/content/tools/stream"
 
if not RUN_Dir:
  RUN_Dir = "tools/stream"
os.makedirs(RUN_Dir, exist_ok=True)

#code to use plugin that was removed from streamlink, remove the comment bellow and set the .py name
#if not os.path.exists(f"{RUN_Dir}/.py"):
#  !cd {RUN_Dir} && wget https://raw.githubusercontent.com/limontec/streamlink/master/src/streamlink/plugins/.py

#@markdown ---
#@markdown ####Type stream url
streamURL =  "https://www.youtube.com/watch?v=0Cs_o3daYR8"  # @param {type:"string"}
#@markdown ---
#@markdown ####Type any name for the stream
name = "K-POP"  # @param {type:"string"}
#@markdown ---
#@markdown ####Settings
#@markdown start stream record
start_record = True #@param {type:"boolean"}
#@markdown try restart record after stream disconnect
loop = True #@param {type:"boolean"}
#@markdown upload video file to your Drive after finish every record
upload_record = True #@param {type:"boolean"}
#@markdown upload all videos in this session to your Drive
upload_all = False #@param {type:"boolean"}
#@markdown ---
#@markdown #####Sites supported: https://streamlink.github.io/plugin_matrix.html
#@markdown ---

def record():
  tempo = time.strftime('_%d-%m-%Y_%H-%M-%S')
  if start_record:
    !cd {RUN_Dir} && streamlink --plugin-dirs {RUN_Dir} {streamURL} best -o {RUN_Dir}/{name}{tempo}.mp4 --retry-streams 30 --retry-max 300
  upload(tempo)

def upload(tempo):
  if(os.path.exists('/content/drive')):
    if (upload_record and os.path.exists(RUN_Dir+'/'+name+tempo+'.mp4')) or loop:
      try:
        if(os.path.exists(RUN_Dir+'/'+name+tempo+'.mp4')):
          print("Copying "+name+tempo+".mp4 to drive...")
          !cd {RUN_Dir} && cp {name}{tempo}.mp4 /content/drive/My\ Drive/stream
        else:
          print("No file.")
      except Exception:
        print("Error!")
    elif upload_all:
      try:  
        print("Copying *.mp4 to drive...")
        os.system('cd '+RUN_Dir+' && cp *.mp4 /content/drive/My\ Drive/stream')
      except Exception:
        print("Error!")
    else:
      print("Nothing copied.")
  else:
    print("You need to mount Gdrive to /content/drive to upload")

record()
while loop:
  tempo = time.strftime('_%d-%m-%Y_%H-%M-%S')
  record()
  clear_output()
